In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from set_env import set_env
set_env()
# Must set SNORKELDB before importing SnorkelSession
from snorkel import SnorkelSession
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
session = SnorkelSession()

In [2]:
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Documents: 400
Sentences: 95656


In [3]:
from snorkel.models import candidate_subclass


# GenePhenoPair = candidate_subclass('GenePhenoPair',['gene', 'pheno'])
PhenoPair = candidate_subclass('Phenotypes',['descriptor', 'entity'])

In [4]:
#all_sents = set()
#docs = session.query(Document).order_by(Document.name).all()
#for i, doc in enumerate(docs):
    #for s in doc.sentences:
        #all_sents.add(s)
import cPickle
with open('small_data/pmcids_400.pkl', 'rb') as f:
    sent_dicts = cPickle.load(f)
train_ids, dev_ids, test_ids = set(sent_dicts['train']), set(sent_dicts['dev']), set(sent_dicts['test'])
all_ids = train_ids.union(dev_ids).union(test_ids)
# 40, 10, 10
train_sents, dev_sents, test_sents, all_sents = set(), set(), set(), set()
train_docs, dev_docs, test_docs = set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()

doc_sents = dict()
for doc_num, doc in enumerate(docs):
    if len(train_docs) >= 40 and len(dev_docs) >= 10 and len(test_docs) >= 10:break
    doc_sents[doc_num] = set()
    for s in doc.sentences:
        all_sents.add(s)
        doc_sents[doc_num].add(s)
        name = doc.name.split('-')[0]
        if name in train_ids:
            train_docs.add(name)
            train_sents.add(s)
        elif name in dev_ids:
            dev_docs.add(name)
            dev_sents.add(s)
        elif name in test_ids:
            test_docs.add(name)
            test_sents.add(s)
        else:
            raise Exception('ID <{0}> not found in any id set'.format(doc.name))

In [5]:
print len(all_sents), len(train_sents), len(dev_sents), len(test_sents)
#print all_sents

24754 19620 2732 2402


In [6]:
#from revised_extraction import PM
from pheno_candidates import PATO, OBO
# from candidate_extraction import PM, GM
from snorkel.candidates import Ngrams, CandidateExtractor


gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=10)
#phenos = Ngrams(n_max=5)
# cand_extractor = CandidateExtractor(GenePhenoPair, 
#                                     [gene_ngrams, pheno_ngrams], [GM, PM],
#                                     symmetric_relations=True, nested_relations=True)
pheno_extractor = CandidateExtractor(PhenoPair, 
                                    [pheno_ngrams, pheno_ngrams], [PATO, OBO],
                                    symmetric_relations=True)

pheno_candidates.py:105: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return [pheno.lower() for pheno in terms if pheno.lower() not in blacklist and len(pheno)>1]
/home/CS341_Arabidopsis/venv/local/lib/python2.7/site-packages/nltk/stem/porter.py:274: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if word[-1] == 's':
/home/snorkel/snorkel/matchers.py:112: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  self.d = frozenset(self._stem(w) for w in list(self.d) if self._stem(w) not in self.blacklist and len(w)>2)


In [22]:
# print "Extracting Candidates..."

# cand_extractor.apply(dev_sents, split=0)
# dev_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==0).all()
# print "Number of dev candidates:", len(dev_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 788


In [25]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(dev_cands, session, annotator_name = 'gold')

NameError: name 'dev_cands' is not defined

In [ ]:
# sv

In [7]:
print "Extracting Candidates..."

pheno_extractor.apply(dev_sents, split=0)
pheno_dev_cands = session.query(PhenoPair).filter(PhenoPair.split==0).all()
print "Number of dev candidates:", len(pheno_dev_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 3222


In [8]:
from snorkel.viewer import SentenceNgramViewer
svp = SentenceNgramViewer(pheno_dev_cands, session, annotator_name = 'gold')

<IPython.core.display.Javascript object>

In [9]:
svp